# Imagens Sintéticas dos dados Simulados

## Autores
Yi-Hao Chen, Sebastian Heinz, Kelle Cruz, Stephanie T. Douglas

## Tradução
Marcos Soares

## Objetivos de aprendizado

- Atribuir astrometria WCS para uma imagem usando ```astropy.wcs``` 
- Construir uma PSF usando ```astropy.modeling.model```
- Convoluir  dados crus com PSF usando ```astropy.convolution```
- Calcular a fração de polarização e ângulo a partir dos dados Stokes I, Q, U
- Sobrepor flechas na imagem.

## Palavras-Chave
modelagem, convolução, coordenadas, WCS, FITS, astronomia de rádio, matplotlib, barra de cores

## Sumário
Nesse tutorial, iremos:

[1. Carregar e examinar os arquivos FITS](#1.-Carregar-e-examinar-os-arquivos-FITS)

[2. Configurar coordenadas de astrometria](#2.-Configurar-coordenadas-de-astrometria)

[3. Preparar uma Point Spread Function (PSF)](#3.-Preparar-uma-Point-Spread-Function-(PSF))

>[3. Como fazer isso sem kernels do astropy](#3.a-Como-fazer-isso-sem-kernels-do-astropy)

[4. Convolver uma imagem com PSF](#4.-Convolver-uma-imagem-com-PSF)

[5. Convolver imagens Stokes Q e U](#5.-Convolver-imagens-Stokes-Q-e-U)

[6. Calcular ângulo e fração de polarização para plote de vetores](#6.-Calcular-ângulo-e-fração-de-polarização-para-plote-de-vetore)

In [ ]:
from astropy.utils.data import download_file
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

from astropy.convolution import Gaussian2DKernel
from astropy.modeling.models import Lorentz1D
from astropy.convolution import convolve_fft

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## 1. Carregar e examinar arquivos FITS

Aqui nós iniciamos com dados de duas dimensões(2D) que foi armazenado em formato FITS a partir de simulações. Temos mapas Stokes I, Q, e U. Iremos primeiramente carregar um arquivo FITS e examinar o cabeçalho.

In [ ]:
file_i = download_file(
    'http://data.astropy.org/tutorials/synthetic-images/synchrotron_i_lobe_0700_150MHz_sm.fits', 
    cache=True)
hdulist = fits.open(file_i)
hdulist.info()

hdu = hdulist['NN_EMISSIVITY_I_LOBE_150.0MHZ']
hdu.header

Podemos ver esse arquivo FITS, que foi criado em [yt](https://yt-project.org/), tem coordenadas x e y em unidades físicas (cm). E queremos converter em coordenadas celestes. Antes de proceder, vamos encontrar o intervalo de dados e plotar um histograma. 

In [ ]:
print(hdu.data.max())
print(hdu.data.min())
np.seterr(divide='ignore') #Suprimir os alertas levantados através dos log10 dos dados com zeros
plt.hist(np.log10(hdu.data.flatten()), range=(-3, 2), bins=100);

Uma vez que saibamos o intervalo dos dados, podemos fazer uma vizualização com o intervalo apropriado (```vmin``` and ```vmax```).

In [ ]:
fig = plt.figure(figsize=(6,12))
fig.add_subplot(111)

# Plotamos numa escala logarítmica e com números pequenos para evitar valores nan. 
plt.imshow(np.log10(hdu.data+1E-3), vmin=-1, vmax=1, origin='lower')

## 2. Configurar coordenadas de astrometria

A partir do cabeçalho, sabemos que os eixos x e y são em centímetros. Contudo, numa observação temos usualmente a Ascenção Reta (RA) e a Declinação(Dec). Para converter unidades físicas para coordenadas celestes, precisaremos supor onde o objeto está localizado, i.e. a distância ao objeto e a RA e Dec central. 

In [ ]:
# Distância ao objeto
dist_obj = 200*u.Mpc

# Temos então os formatos respectivos RA em hh:mm:ss e DEC em dd:mm:ss. 
# Iremos usar o Skycoord para converter elas em graus posteriormente.
ra_obj = '19h59m28.3566s'
dec_obj = '+40d44m02.096s'

Aqui convertemos a escala de pixel de cm para grau dividindo a distância ao objeto.

In [ ]:
cdelt1 = ((hdu.header['CDELT1']*u.cm/dist_obj.to('cm'))*u.rad).to('deg')
cdelt2 = ((hdu.header['CDELT2']*u.cm/dist_obj.to('cm'))*u.rad).to('deg')
print(cdelt1, cdelt2)

Use ```astropy.wcs.WCS``` para preparar o cabeçalho FITS.

In [ ]:
w = WCS(naxis=2)

# Coordenada de pixel de referência
w.wcs.crpix = [hdu.data.shape[0]/2,hdu.data.shape[1]/2]

# Tamanhos dos pixels em graus
w.wcs.cdelt = [-cdelt1.base, cdelt2.base]

# Convertendo ra e dec em graus
c = SkyCoord(ra_obj, dec_obj)
w.wcs.crval = [c.ra.deg, c.dec.deg]

# As unidades dos eixos são em graus
w.wcs.cunit = ['deg', 'deg']

Agora podemos converter as coordenadas WCS no cabeçalho e atualizar o hdu com o update.

In [ ]:
wcs_header = w.to_header()
hdu.header.update(wcs_header)

Vamos dar uma olhada no cabeçalho. ```CDELT1```, ```CDELT2```, ```CUNIT1```, ```CUNIT2```, ```CRVAL1```, e ```CRVAL2``` são em coordenadas celestes agora.

In [ ]:
hdu.header

In [ ]:
wcs = WCS(hdu.header)

fig = plt.figure(figsize=(6,12))
fig.add_subplot(111, projection=wcs)
plt.imshow(np.log10(hdu.data+1e-3), vmin=-1, vmax=1, origin='lower')
plt.xlabel('RA')
plt.ylabel('Dec')

Agora temos as coordenadas celestes para a imagem!

## 3. Preparar uma Point Spread Function (PSF)

Simples PSFs são incluídas em ```astropy.convolution.kernel```. Aqui usaremos ```astropy.convolution.Gaussian2DKernel```.
Primeiro precisamos configurar a resolução do telescópio. Para uma Gaussiana 2D, podemos calcular sigma em  pixels usando nossa palavra chave da escala de pixel  ```cdelt2``` que está acima.

In [ ]:
# Assuma que nosso telescópio tem uma resolução de 1 arcsecond
telescope_resolution = 1*u.arcsecond

# Calcule o sigma em pixels. 
# Desde que cdelt está em graus, podemos usar _.to('deg')
sigma = telescope_resolution.to('deg')/cdelt2

In [ ]:
# Por padrão, o kernel Gaussiano irá para 4 sigma 
# em cada direção
psf = Gaussian2DKernel(sigma)

# Vamos dar uma olhada:
plt.imshow(psf.array.value)

## 3.a Como fazer isso sem kernels do astropy

Talvez seu PSF é mais complicado. Aqui está um jeito alternativo para isso, usando um Lorentziana 2D.

In [ ]:
# Configure a grade FWHM e psf 
telescope_resolution = 1*u.arcsecond
gamma = telescope_resolution.to('deg')/cdelt2
x_grid = np.outer(np.linspace(-gamma*4,gamma*4,int(8*gamma)),np.ones(int(8*gamma)))
r_grid = np.sqrt(x_grid**2 + np.transpose(x_grid**2))
lorentzian = Lorentz1D(fwhm=2*gamma)

# Expulse uma PSF simétrica azimutalmente.
lorentzian_psf = lorentzian(r_grid)

# Normalização
lorentzian_psf /= np.sum(lorentzian_psf)

# Vamos dar uma olhada de novo:
plt.imshow(lorentzian_psf.value, interpolation='none')

## 4. Convolver uma imagem com PSF

Aqui usamos ```astropy.convolution.convolve_fft``` para convolver imagens. Essa rotina usa transformada de fourier para cálculos mais rápidos. Especialmente se nossos dados têm tamanho $2^n$, o que os torna particularmente rápidos. Usando um fft, contudo, causa efeitos de contorno. Iremos então precisar especificar como queremos lidar com as bordas. Aqui escolhemos "embrulhar" os dados. O que significa torná-los periódicos.

In [ ]:
convolved_image = convolve_fft(hdu.data, psf, boundary='wrap') 

In [ ]:
#Ponha uma psf no canto da imagem
delta_x_psf=100 # Número de pixels das arestas 
xmin, xmax = -psf.shape[1]-delta_x_psf, -delta_x_psf
ymin, ymax = delta_x_psf, delta_x_psf+psf.shape[0]
convolved_image[xmin:xmax, ymin:ymax] = psf.array/psf.array.max()*10

Agora dê uma olhada na imagem convolvida.

In [ ]:
wcs = WCS(hdu.header)
fig = plt.figure(figsize=(8,12))
i_plot = fig.add_subplot(111, projection=wcs)
plt.imshow(np.log10(convolved_image+1e-3), vmin=-1, vmax=1.0, origin='lower')#, cmap=plt.cm.viridis)
plt.xlabel('RA')
plt.ylabel('Dec')
plt.colorbar()

## 5. Convolver imagens Stokes Q e U

In [ ]:
hdulist.info()

In [ ]:
file_q = download_file(
    'http://data.astropy.org/tutorials/synthetic-images/synchrotron_q_lobe_0700_150MHz_sm.fits', 
    cache=True)
hdulist = fits.open(file_q)
hdu_q = hdulist['NN_EMISSIVITY_Q_LOBE_150.0MHZ']

file_u = download_file(
    'http://data.astropy.org/tutorials/synthetic-images/synchrotron_u_lobe_0700_150MHz_sm.fits', 
    cache=True)
hdulist = fits.open(file_u)
hdu_u = hdulist['NN_EMISSIVITY_U_LOBE_150.0MHZ']

# Atualize o cabeçalho com o wcs_header que criamos anteriormente
hdu_q.header.update(wcs_header)
hdu_u.header.update(wcs_header)

# Convolver as imagens com a psf
convolved_image_q = convolve_fft(hdu_q.data, psf, boundary='wrap')
convolved_image_u = convolve_fft(hdu_u.data, psf, boundary='wrap')

Agora vamos plotar as imagens Q e U

In [ ]:
wcs = WCS(hdu.header)
fig = plt.figure(figsize=(16,12))
fig.add_subplot(121, projection=wcs)
plt.imshow(convolved_image_q, cmap='seismic', vmin=-0.5, vmax=0.5, origin='lower')#, cmap=plt.cm.viridis)
plt.xlabel('RA')
plt.ylabel('Dec')
plt.colorbar()

fig.add_subplot(122, projection=wcs)
plt.imshow(convolved_image_u, cmap='seismic', vmin=-0.5, vmax=0.5, origin='lower')#, cmap=plt.cm.viridis)

plt.xlabel('RA')
plt.ylabel('Dec')
plt.colorbar()

## 6. Calcular ângulo e fração de polarização para plote de vetores

Note que rotacionando mapas Stokes Q e I requer sinais de mudanças de ambos. Assumimos aqui que os mapas Q e U foram calculados definindo o eixo y/declinação como vertical, tal que Q seja positivo para vetores de polarização ao longo do eixo x/ascenção reta.

In [ ]:
# Primeiro, plotamos o plano de fundo da imagem
fig = plt.figure(figsize=(8,16))
i_plot = fig.add_subplot(111, projection=wcs)
i_plot.imshow(np.log10(convolved_image+1e-3), vmin=-1, vmax=1, origin='lower')

# Intervalos dos eixos
xx0, xx1 = i_plot.get_xlim()
yy0, yy1 = i_plot.get_ylim()

# Fator de bin
factor = [64, 66]

# número de pontos em cada eixo rebinados 
nx_new = convolved_image.shape[1] // factor[0]
ny_new = convolved_image.shape[0] // factor[1]

# Essas são as posições dos vetores
X,Y = np.meshgrid(np.linspace(xx0,xx1,nx_new,endpoint=True),
                  np.linspace(yy0,yy1,ny_new,endpoint=True))

# binar os dados 
I_bin = convolved_image.reshape(nx_new, factor[0], ny_new, factor[1]).sum(3).sum(1)
Q_bin = convolved_image_q.reshape(nx_new, factor[0], ny_new, factor[1]).sum(3).sum(1)
U_bin = convolved_image_u.reshape(nx_new, factor[0], ny_new, factor[1]).sum(3).sum(1)

# ângulo de polarização
psi = 0.5*np.arctan2(U_bin, Q_bin)

# fração de polarização
frac = np.sqrt(Q_bin**2+U_bin**2)/I_bin

# máscara para área de sinal baixo
mask = I_bin < 0.1

frac[mask] = 0
psi[mask] = 0

pixX = frac*np.cos(psi) # Vetor-X 
pixY = frac*np.sin(psi) # Vetor-Y

# argumentos palavra-chave para plote de vetores
quiveropts = dict(headlength=0, headwidth=1, pivot='middle')
i_plot.quiver(X, Y, pixX, pixY, scale=8, **quiveropts)

## Exercício

### Converter  as unidades dos dados de Jy/arcsec^2 para Jy/beam

A intensidade dos dados é dada em unidades de Jy/arcsec^2. Dados observasionais têm usualmente a unidade de intensidade em Jy/beam. Assumindo um tamanho beam ou pegando a psf que criamos anteriormente, você pode converter os dados em Jy/beam.